In [20]:
import os
import pandas as pd
from scipy.stats import kendalltau

def compute_kendall_for_all_methods():
    """
    Computes Kendall’s Tau correlation coefficient between two ranking results 
    for different decision-making methods.
    """
    # List of ranking methods to compare
    methods = ["WSM", "WPM", "WASPAS", "TOPSIS", "VIKOR", "PROMETHEE"]
    
    results = {}
    
    for method in methods:
        # Define filenames for the two ranking versions: ahp_... and ahp2_...
        file1 = f"static/results/ahp_{method}_full_ranking.xlsx"
        file2 = f"static/results/ahp2_{method}_full_ranking.xlsx"
        
        # Check if both files exist
        if not (os.path.exists(file1) and os.path.exists(file2)):
            print(f"[Warning] One of the files for {method} does not exist. Skipping...")
            continue
        
        # Read both Excel files
        df1 = pd.read_excel(file1)
        df2 = pd.read_excel(file2)
        
        # Create a dictionary of rankings from each file (0th row = rank 1, 1st row = rank 2, etc.)
        rank1 = {row["model_name"]: i + 1 for i, row in df1.iterrows()}
        rank2 = {row["model_name"]: i + 1 for i, row in df2.iterrows()}
        
        # Identify common models in both rankings
        common_models = set(rank1.keys()).intersection(set(rank2.keys()))
        
        # Extract corresponding rankings
        list_rank1 = [rank1[m] for m in common_models]
        list_rank2 = [rank2[m] for m in common_models]
        
        # Compute Kendall’s Tau only if enough data is available
        if len(list_rank1) < 2:
            print(f"[Warning] Not enough common models to compute Kendall’s Tau for {method}")
            continue
        
        tau, p_value = kendalltau(list_rank1, list_rank2)
        
        results[method] = (tau, p_value)
    
    return results

In [21]:
kendall_results = compute_kendall_for_all_methods()
print("Kendall’s Tau results between ahp1 and ahp2 for each method:")
for method, (tau, pval) in kendall_results.items():
    print(f"  {method}: Tau = {tau:.4f}")

Kendall’s Tau results between ahp1 and ahp2 for each method:
  WSM: Tau = 0.9188
  WPM: Tau = 0.9493
  WASPAS: Tau = 1.0000
  TOPSIS: Tau = 0.9599
  VIKOR: Tau = 0.8236
  PROMETHEE: Tau = 0.9188
